In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
#Install JAVA/Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
!pip install -q findspark
!pip install pyspark

In [0]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import shutil
folder = "/content/gdrive/My Drive/Colab Notebooks/COLAB DATASETS/foursquare_logs"
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

FileNotFoundError: ignored

In [0]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

In [0]:
def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

In [0]:
# create spark configuration
conf = SparkConf()
conf.setAppName("FoursquareStreamApp")

# create spark context with the above configuration
sc = SparkContext.getOrCreate(conf=conf)
sc.setLogLevel("ERROR")

# create the Streaming Context from the above spark context with 
# interval size 10 seconds
ssc = StreamingContext(sc,10)

# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_FoursquareApp")

# read data from drive
dataStream = ssc.textFileStream("/content/gdrive/My Drive/Colab Notebooks/COLAB DATASETS/foursquare_logs/")

In [0]:
visitor_counts = dataStream.map(lambda x: (x.split(",")[0], int(x.split(",")[1]))).reduceByKey(lambda a, b: a + b)

runningCounts = visitor_counts.updateStateByKey(updateFunction)

runningCounts.pprint()

# start the streaming computation
ssc.start()

# wait for the streaming to finish
ssc.awaitTermination()

-------------------------------------------
Time: 2019-11-19 20:46:40
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:46:50
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:47:00
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:47:10
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:47:20
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:47:30
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:47:40
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:47:50
-------------------------------------------

-------------------------------------------
Time: 2019-11-19 20:48:00
----------

KeyboardInterrupt: ignored

In [0]:
ssc.stop()